##**Conectar com o Google**

In [120]:
#Conectar com o Google Drive

from google.colab import drive
drive.mount('/content/drive')

#Conectando o ambiente ao Google Drive

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##**Importar bibliotecas**

In [121]:
#@title Puxar Base de Dados

#Importar bibliotecas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, date
import statistics
from statistics import StatisticsError
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif, mutual_info_classif


#Importando bibliotecas necessárias

uri = '/content/drive/Shareddrives/Colab_Grupo2/Colab/Data/dados_testev2.csv'

data = pd.read_csv(uri)


#Declarando caminho do conjunto de dados e instanciando um dataframe

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (22,24,25,75,76,77,78,89,90,91,92) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


##**Tratamento de dados**

In [122]:
#@title Algoritmo de tratamento de dados

#@markdown Execute o algoritmo para corrigir a sua base de dados

data_columns = data.columns
data_columns_date = ["dob", "date_last_fu", "dtrecidiva", "data_2"]
data_columns_string_to_numbers = ["rec01", "rec02", "rec03", "rec04", "meta01", "meta02", "meta03", "meta04"]
data_columns_str_to_percentage = ["er_percentage", "progesteron_perct"]

#Delcarando as variáveis de armazenamento de lista de colunas a serem tratadas

incorrect_values = {
    "0": ["neg", "NEG", "inconclusivo"],
    "10": ["1-10"],
    "98": ["980"],
    "100": ["1000"],
    "70": ["20-70","70%"]
}

#Variável de erros de digitação a serem substituidos pelas chaves

for x in range(len(data_columns_str_to_percentage)):
  data[data_columns_str_to_percentage[x]] = data[data_columns_str_to_percentage[x]].astype(str)
  for key, values in incorrect_values.items():
    for especific_values in values:
      data[data_columns_str_to_percentage[x]] = data[data_columns_str_to_percentage[x]].str.replace(especific_values,key)
  data[data_columns_str_to_percentage[x]] = data[data_columns_str_to_percentage[x]].str.lstrip('nan')
  data[data_columns_str_to_percentage[x]] = pd.to_numeric(data[data_columns_str_to_percentage[x]], downcast="integer")

#Esse laço é capaz de corrigir os valores das colunas listadas especialmente por erros de digitação, transformando-os em int


for x in range(len(data_columns_string_to_numbers)):
  data[data_columns_string_to_numbers[x]] = data[data_columns_string_to_numbers[x]].astype(str)
  data[data_columns_string_to_numbers[x]] = data[data_columns_string_to_numbers[x]].str.lstrip('C')
  data[data_columns_string_to_numbers[x]] = data[data_columns_string_to_numbers[x]].str.lstrip('nan')
  data[data_columns_string_to_numbers[x]] = pd.to_numeric(data[data_columns_string_to_numbers[x]], downcast="integer")

#Esse laço é capaz de corrigir os valores das colunas listadas especialmente por erros de digitação, transformando-os em int

for x in range(len(data_columns)):
  data[data_columns[x]].replace(np.inf, np.nan, inplace=True)

#Esse laço é capaz de substituir valores inf por nulos em todo o data frame

for x in range(len(data_columns_date)):
  data[data_columns_date[x]] = pd.to_datetime(data[data_columns_date[x]])

#Esse laço é capaz de transformar strings em forma de datas

#NOVO

def specif_sort(x):
  data_filter_1 = x.loc[x['redcap_repeat_instrument'] != 'dados_antropometricos']
  data_filter_2 = x.loc[x['redcap_repeat_instrument'] == 'dados_antropometricos']
  data_filter_2 = data_filter_2.sort_values(by='data_2')
  return pd.concat([data_filter_1, data_filter_2])

data = data.groupby(by='record_id').apply(specif_sort)

data.reset_index(drop=True, inplace=True)

#Essa função é capaz de ordenar todas as linhas que possuem valor na coluna data_2 por id
#e ainda sim manter esses valores ordenados mesmo para linhas em que data_2 é nula

#data.reset_index(drop=True, inplace=True)

In [123]:
#@title Algoritmo de exclusão de colunas

#@markdown Informe a coluna a ser deletada

default_columns= True #@param {type:"boolean"}
column = 'partos' #@param ["record_id", "redcap_repeat_instrument", "redcap_repeat_instance", "dob", "escolari", "level_education", "race", "pregnancy_history", "partos", "abortion", "breast_feeding", "breast_feeding_time", "menarche", "period", "bmi", "contraceptive_method___1", "contraceptive_method___2", "contraceptive_method___3", "contraceptive_method___4", "contraceptive_method___999", "contraceptive", "hormone_therapy", "hormone_therapy_tasy", "tempo_rep_hormo", "tempo_rep_hormo_tasy", "antec_fam_cancer_mama", "tobaco", "alcohol", "tobaco_type", "alcohol_type___1", "alcohol_type___2", "alcohol_type___3", "family_cancer", "familial_degree___1", "familial_degree___2", "familial_degree___3", "family_cancer_id___1", "family_cancer_id___2", "family_cancer_id___3", "family_cancer_id___4", "family_cancer_id___5", "family_cancer_id___6", "mother_tumor", "sister_n", "daughter_n", "sister_tumor_1", "daughter_tumor_1", "birads", "benign", "primary_diganosis", "diff_tubular", "nuclear_grade", "mitotic_index", "histological_grade", "grau_hist", "multicentricidade", "multifocal", "tumor_margin", "er_ihc", "pr_ihc", "tumor_subtype", "progesteron_perct", "er_percentage", "her2_ihc", "her2_fish", "ki67_perc", "date_last_fu", "follow_up_days", "ultinfo", "recidive", "dtrecidiva", "follow_up_days_recidive", "reclocal", "recregio", "recdist", "rec01", "rec02", "rec03", "rec04", "trathosp", "topo", "basediag", "desctopo", "morfo", "tumor_stage", "t", "n", "m", "meta01", "meta02", "meta03", "meta04", "tipotrat", "treatment", "chemotherapy", "hormontx", "hormontxtype", "tipo_qt_neo_her2", "radioterapia", "hormone_therapy_yn", "seguim_qt_realizou", "seguim_qt", "data_2", "weight", "height", "gli", "hb", "ht", "leuco", "neutro", "linfocitos", "ureia", "creat", "hiv"]

#Este formulário armazena se o usuário quer utilizar as colunas de exclusão padrão ou 
#selecionar aquela que ele quer excluir

#NOVO

data['data_2_first']=data['data_2'].copy()
data['data_2_last']=data['data_2'].copy()

#Criação de colunas novas de data de consulta item b

default = ["desctopo","descmorfo","hormone_therapy_tasy","tempo_rep_hormo_tasy","redcap_repeat_instrument","redcap_repeat_instance", "dtrecidiva", "data_2"]

for x in data.columns:
  if data[x].notna().sum() < 1:
    default.append(x)

default_data_1 =["dob", "escolari", "level_education", "race", "pregnancy_history", "partos", "abortion", "breast_feeding", "breast_feeding_time", "menarche", "period", "contraceptive_method___1", "contraceptive_method___2", "contraceptive_method___3", "contraceptive_method___4", "contraceptive_method___999", "contraceptive", "hormone_therapy", "tempo_rep_hormo", "antec_fam_cancer_mama", "tobaco", "alcohol", "alcohol_type___1", "alcohol_type___2", "alcohol_type___3", "family_cancer", "familial_degree___1", "familial_degree___2", "familial_degree___3", "family_cancer_id___1", "family_cancer_id___2", "family_cancer_id___3", "family_cancer_id___4", "family_cancer_id___5", "family_cancer_id___6", "sister_n", "daughter_n", "birads", "benign", "primary_diganosis", "nuclear_grade", "mitotic_index", "histological_grade", "grau_hist", "multicentricidade", "multifocal", "tumor_margin", "er_ihc", "pr_ihc", "tumor_subtype", "progesteron_perct", "er_percentage", "her2_ihc", "her2_fish", "ki67_perc", "date_last_fu", "follow_up_days", "ultinfo", "recidive", "follow_up_days_recidive", "reclocal", "recregio", "recdist", "rec01", "rec02", "rec03", "rec04", "trathosp", "topo", "basediag", "morfo", "tumor_stage", "t", "n", "m", "meta01", "meta02", "meta03", "meta04", "treatment", "chemotherapy", "tipo_qt_neo_her2", "radioterapia", "hormone_therapy_yn", "seguim_qt_realizou", "seguim_qt", "data_2_first", "data_2_last"]

#Vetor de colunas que usam método de primeiro valor e ultimo valor

default_data_2 =["weight", "height","bmi", "tobaco_type", "mother_tumor","sister_tumor_1", "daughter_tumor_1", "diff_tubular", "data_2_last"]

#Vetor de colunas que usam método ultimo valor, flutuacao, média, moda e devio pardrão

default_data_3 = ["weight", "height","bmi", "tobaco_type", "mother_tumor","sister_tumor_1", "daughter_tumor_1", "diff_tubular", "dob", "escolari", "level_education", "race", "pregnancy_history", "partos", "abortion", "breast_feeding", "breast_feeding_time", "menarche", "period", "contraceptive_method___1", "contraceptive_method___2", "contraceptive_method___3", "contraceptive_method___4", "contraceptive_method___999", "contraceptive", "hormone_therapy", "tempo_rep_hormo", "antec_fam_cancer_mama", "tobaco", "alcohol", "alcohol_type___1", "alcohol_type___2", "alcohol_type___3", "family_cancer", "familial_degree___1", "familial_degree___2", "familial_degree___3", "family_cancer_id___1", "family_cancer_id___2", "family_cancer_id___3", "family_cancer_id___4", "family_cancer_id___5", "family_cancer_id___6", "sister_n", "daughter_n", "birads", "benign", "primary_diganosis", "nuclear_grade", "mitotic_index", "histological_grade", "grau_hist", "multicentricidade", "multifocal", "tumor_margin", "er_ihc", "pr_ihc", "tumor_subtype", "progesteron_perct", "er_percentage", "her2_ihc", "her2_fish", "ki67_perc", "date_last_fu", "follow_up_days", "ultinfo", "recidive", "follow_up_days_recidive", "reclocal", "recregio", "recdist", "rec01", "rec02", "rec03", "rec04", "trathosp", "topo", "basediag", "morfo", "tumor_stage", "t", "n", "m", "meta01", "meta02", "meta03", "meta04", "treatment", "chemotherapy", "tipo_qt_neo_her2", "radioterapia", "hormone_therapy_yn", "seguim_qt_realizou", "seguim_qt", "data_2_first"]

#Vetor de colunas que usam método primeiro valor, flutuacao, média, moda e devio pardrão

for x in default:
  if x in default_data_1:
    default_data_1.remove(x)
  if x in default_data_2:
    default_data_2.remove(x)
  if x in default_data_3:
    default_data_3.remove(x)


#Lista de colunas padrões a serem excluidas

if default_columns == True:
  data = data.drop(columns = default)

  data_1 = data.copy()
  data_2 = data.copy()
  data_3 = data.copy()

  data_1 = data_1.drop(columns = default_data_1)
  data_2 = data_2.drop(columns = default_data_2)
  data_3 = data_3.drop(columns = default_data_3)

else:
  del data[column]

#Essa condição verifica se o usuário que usar as colunas padrões, caso verdadeiro, percorre
#a lista de colunas a serem excluídas


In [124]:
#@title Algoritmo de preenchimento de nulos

default_columns= True #@param {type:"boolean"}

#@markdown Informe o método de preenchimento de valores nulos
method = 'media' #@param ["mediana", "moda", "media",  "primeiro_Registro"]

#@markdown Informe as coluna que você deseja implementar este método
column = 'period' #@param ["record_id", "redcap_repeat_instrument", "redcap_repeat_instance", "dob", "escolari", "level_education", "race", "pregnancy_history", "partos", "abortion", "breast_feeding", "breast_feeding_time", "menarche", "period", "bmi", "contraceptive_method___1", "contraceptive_method___2", "contraceptive_method___3", "contraceptive_method___4", "contraceptive_method___999", "contraceptive", "hormone_therapy", "hormone_therapy_tasy", "tempo_rep_hormo", "tempo_rep_hormo_tasy", "antec_fam_cancer_mama", "tobaco", "alcohol", "tobaco_type", "alcohol_type___1", "alcohol_type___2", "alcohol_type___3", "family_cancer", "familial_degree___1", "familial_degree___2", "familial_degree___3", "family_cancer_id___1", "family_cancer_id___2", "family_cancer_id___3", "family_cancer_id___4", "family_cancer_id___5", "family_cancer_id___6", "mother_tumor", "sister_n", "daughter_n", "sister_tumor_1", "daughter_tumor_1", "birads", "benign", "primary_diganosis", "diff_tubular", "nuclear_grade", "mitotic_index", "histological_grade", "grau_hist", "multicentricidade", "multifocal", "tumor_margin", "er_ihc", "pr_ihc", "tumor_subtype", "progesteron_perct", "er_percentage", "her2_ihc", "her2_fish", "ki67_perc", "date_last_fu", "follow_up_days", "ultinfo", "recidive", "dtrecidiva", "follow_up_days_recidive", "reclocal", "recregio", "recdist", "rec01", "rec02", "rec03", "rec04", "trathosp", "topo", "basediag", "morfo", "tumor_stage", "t", "n", "m", "meta01", "meta02", "meta03", "meta04", "tipotrat", "treatment", "chemotherapy", "hormontx", "hormontxtype", "tipo_qt_neo_her2", "radioterapia", "hormone_therapy_yn", "seguim_qt_realizou", "seguim_qt", "data_2", "weight", "height", "gli", "hb", "ht", "leuco", "neutro", "linfocitos", "ureia", "creat", "hiv"]
 
#Este formulário armazena se o usuário quer utilizar as colunas de preenchimento padrão ou 
#selecionar aquela com o método escolhido

columns = {
    "flutuacao": ["weight"],
    "media": ["height"],
    "moda": ["tobaco_type", "mother_tumor","sister_tumor_1", "daughter_tumor_1", "diff_tubular"],
    "desvio_Pradao": ["bmi"]
}

#Declarando em biblioteca as colunas com seus respectivos métodos

functions = locals()

def func_lregister(mini_data):
    lista_valores = []
    for c in mini_data.columns:
        index_m = mini_data[c].last_valid_index()
        if index_m != None:
            valor_nao_nulo = mini_data[c].loc[index_m]
            lista_valores.append(valor_nao_nulo)
        else:
            lista_valores.append(np.nan)
      
    return pd.DataFrame(data=[lista_valores], columns=mini_data.columns)

def func_fregister(mini_data):
    lista_valores = []
    for c in mini_data.columns:
        index_m = mini_data[c].first_valid_index()
        if index_m != None:
            valor_nao_nulo = mini_data[c].loc[index_m]
            lista_valores.append(valor_nao_nulo)
        else:
            lista_valores.append(np.nan)
      
    return pd.DataFrame(data=[lista_valores], columns=mini_data.columns)

#Essa função utilizada nos metodos de primeiro e ultimo registro e basicamente, identificam os primeiros
#e ultimos valores de cada coluna em realação a cada ID pello groupby da função anterior que chama esta


def flutuacao(column):
  def func(x, column):
    y = x[column].dropna()
    if len(y) > 0:
      x0 = y.iloc[0]
      x1 = y.iloc[-1]
      total = (x1 - x0)/ x0
      return total
    return np.nan
  dfflutuacao = data_1.groupby(by='record_id').apply(lambda x: func(x,column))

  for z in data_1["record_id"]:
    data_1.loc[data_1['record_id'] == z, column] = dfflutuacao.loc[z]
    
  return data_1

# Função que vai realizar a flutuação 

#Essa função substitui os valores da coluna  utilizando o método de mediana dos valores presentes

def moda(column):
  def func(x,column):
    try:
      y = x[column].dropna().to_numpy()
      x[column] = statistics.mode(y)  
    except IndexError:
      pass
    except StatisticsError:
      pass
    return x
  
  return data_1.groupby(by=["record_id"]).apply(lambda x: func(x,column))

#Essa função substitui os valores da coluna  utilizando o método de moda dos valores presentes

def media(column):
  def func(x,column):
    try:
      x[column] = x[column].dropna().mean()
    except IndexError:
      pass
      
    return x

  return data_1.groupby(by=["record_id"]).apply(lambda x: func(x,column))

#Essa função substitui os valores da coluna  utilizando o método de média dos valores presentes

def primeiro_Registro():
  return data_2.groupby(by=['record_id']).apply(func_fregister)

def ultimo_Registro():
  return data_3.groupby(by=['record_id']).apply(func_lregister)

#Essa função substitui os valores da coluna  utilizando o método de primeiro valor presente na coluna

def desvio_Pradao(column):
  def func(x,column):
    try:
      x[column] = x[column].std()
    except IndexError:
      pass
    return x

  return data_1.groupby(by=["record_id"]).apply(lambda x: func(x,column))


if default_columns == True:
  for method, column in columns.items():
    for especif_column in column:
      data_1 = functions[f'{method}'](especif_column)
  data_2 = primeiro_Registro()
  data_3 = ultimo_Registro()
  #NOVO

else:
  data_1 = functions[f'{method}']()

  #Essa condição verifica se o usuário quer executar o preenchimento de colunas padrões,
  #caso verdadeiro, executa laço dentro de laço que percorre os métodos e colunas a serem executadas



In [125]:
#@title Algoritmo de individualização de IDs

#@markdown Execute o algoritmo para individualizar sua base de dados

data_1.drop_duplicates(subset='record_id', inplace = True)

#Exclusão de IDs duplicados

#NOVO

data_2.reset_index(drop=True, inplace=True)

data_3.reset_index(drop=True, inplace=True)

#Reset do índice dos dataframes

#NOVO

data_1 = data_1.merge(data_2, on="record_id")

#Merge de colunas pelo id

data = data_1.merge(data_3, on="record_id")

#Merge de colunas pelo id

del data['record_id']

#Essa célula executa a deleção de linhas duplicadas pelo valor id e consequentemente deleta a coluna de IDs


In [126]:
#@title Algoritmo de criação de colunas

#@markdown Execute o algoritmo para criar as colunas nescessárias

#NOVO

today = date.today()
ages_1 = []
ages_2 = []
ages_3 = []

#Declarando variáveis de armazenamento e de data atual

for x in range(len(data)):
  if data["ultinfo"][x] > 2:
    ages_1.append((pd.DatetimeIndex(data['date_last_fu'])[x].year - pd.DatetimeIndex(data['dob'])[x].year - ((pd.DatetimeIndex(data['date_last_fu'])[x].month, pd.DatetimeIndex(data['date_last_fu'])[x].day) < (pd.DatetimeIndex(data['dob'])[x].month, pd.DatetimeIndex(data['dob'])[x].day)))*365.2425)
  else:
    ages_1.append((today.year - pd.DatetimeIndex(data['dob'])[x].year - ((today.month, today.day) < (pd.DatetimeIndex(data['dob'])[x].month, pd.DatetimeIndex(data['dob'])[x].day)))*365.2425)

#Este laço extrai informação de idade da data de nascimento caso eles esteja vivo ou morto para cada id

for x in range(len(data)):
  ages_2.append((pd.DatetimeIndex(data['date_last_fu'])[x].year - pd.DatetimeIndex(data['data_2_first'])[x].year - ((pd.DatetimeIndex(data['date_last_fu'])[x].month, pd.DatetimeIndex(data['date_last_fu'])[x].day) < (pd.DatetimeIndex(data['data_2_first'])[x].month, pd.DatetimeIndex(data['data_2_first'])[x].day)))*365.2425)

#Este laço extrai informação de tempo de tratamento da data de ultima informação para cada id

for x in range(len(data)):
  ages_3.append((pd.DatetimeIndex(data['data_2_last'])[x].year - pd.DatetimeIndex(data['data_2_first'])[x].year - ((pd.DatetimeIndex(data['data_2_last'])[x].month, pd.DatetimeIndex(data['data_2_last'])[x].day) < (pd.DatetimeIndex(data['data_2_last'])[x].month, pd.DatetimeIndex(data['data_2_last'])[x].day)))*365.2425)

#Este laço extrai informação de tempo de realização de exames da ultima e primeira data de consulta para cada id

data['age']=ages_1
data['treatment_time']=ages_2
data['exam_time']=ages_3

#Criação das novas colunas item b

del data['date_last_fu']
del data["dob"]
del data["data_2_first"]
del data["data_2_last"]

#Deleção das coluna não necessárias neste momento

default_columns = ["age", "exam_time","treatment_time"]

for x in default_columns:
  data.loc[(data[x] < 0),x] = np.nan

#Este laço substitui valores menores que 0 por nulo

data = data.dropna(subset=['follow_up_days'])

#Excluindo os valores nulos na coluna target do modelo

##**Predição de dados nulos**

In [127]:
#@title Algoritmo de conversão de dados string para numéricos

#@markdown Execute o algoritmo para corrigir colunas em texto

le = preprocessing.LabelEncoder()

default_columns = ["antec_fam_cancer_mama"]

for x in default_columns:
  data[x] = le.fit_transform(data[x])

#Este laço percorre o vetor aplicando a função label encoder para transformação de strings
#em números


In [128]:
#@title Algoritmo de particionamento de dataframe

#@markdown Execute o algoritmo para dividir os dados em base de dados para aplicação de método regressivo

default_data =["bmi", "weight", "height", "partos", "breast_feeding_time", "menarche", "tempo_rep_hormo", "sister_n", "daughter_n", "birads", "nuclear_grade", "mitotic_index", "grau_hist", "histological_grade", "progesteron_perct", "er_percentage", "ki67_perc", "follow_up_days", "follow_up_days_recidive", "age", "treatment_time", "exam_time"]


data_1=data.drop(default_data, axis=1).copy()


#Deleção das colunas indicadas no vetor em data_1



In [129]:
#@title Algoritmo de colunas por dados nulos

#@markdown Execute o algoritmo para verificar as colunas com nulos do menor para o maior

percent_missing = data_1.isnull().sum() * 100 / len(data_1)
missing_value_data_1 = pd.DataFrame({'column_name': data_1.columns,
                                 'percent_missing': percent_missing})

#Algoritmo cria um vetor com a porcentagem de nulos para cada coluna e cria um
#novo dataframe com esses valores

missing_value_data_1 = missing_value_data_1.sort_values(by='percent_missing')

#Algoritmo ordena o dataframe pelo valor da porcentagem de nulos



In [130]:
#@title Algoritmo de criação de dicionário

#@markdown Execute o algoritmo para criar o dicionário de  valores a serem executados no modelo preditivo de classificação

columns_dict_1 = []
columns_dict_2 = []
default_data_1 = []
dropna_columns = []
values = []
default_columns = {}

#Criação de vetores e dicionário

for x in range(len(missing_value_data_1)):
  if missing_value_data_1["percent_missing"][x] == 0:
    columns_dict_1.append(missing_value_data_1["column_name"][x])
  elif missing_value_data_1["percent_missing"][x] <= 3:
    dropna_columns.append(missing_value_data_1["column_name"][x])
  elif missing_value_data_1["percent_missing"][x] <= 20:
    columns_dict_2.append(missing_value_data_1["column_name"][x])
  else:
    default_data_1.append(missing_value_data_1["column_name"][x])

data_1 = data_1.drop(default_data_1, axis=1).copy()

#Este laço verifica os valores percentuais igual a 0 e armazena em um vetor e 
#os outros armazena em ordem em outro vetor

count = 0

values = columns_dict_1.copy()

for x in columns_dict_2:
  default_columns[x] = list(values)
  values.append(x)

#Este laço adicona no dicionário o vetor de colunas e a chave respectiva a este vetor
#para que seja possível percorrer cada predição com o conjunto de colunas para aquele target


In [131]:
#@title Algoritmo de predição de dados para todas as colunas classificativas

#@markdown Execute o algoritmo para predizer e repopular a base restante com nulos por resultados da predição classificativa

for x in default_columns.keys():

  data_1_current = data_1.copy()
  data_1_current = data_1_current[default_columns.get(x)+[x]]

  data_nan_1 = data_1_current[data_1_current[x].isnull()]
  data_not_nan_1 = data_1_current[data_1_current[x].notnull()]

    
  clf = RandomForestClassifier(max_depth=2, random_state=0)
  
  clf.fit(data_not_nan_1[default_columns.get(x)], data_not_nan_1[x])

  data_nan_1[x] = clf.predict(data_nan_1[default_columns.get(x)])

  data_1[x] = pd.concat([data_nan_1[x], data_not_nan_1[x]])


#Algoritmo de predição por classificação para a chave x do dicionário com a 
#respectiva chave como base da entrada do modelo



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [132]:
#@title Algoritmo de colunas por dados nulos

#@markdown Execute o algoritmo para verificar as colunas do modelo regressivo com nulos do menor para o maior

for x in data_1.columns:
  data[x] = data_1[x].copy()

percent_missing = data.isnull().sum() * 100 / len(data)
missing_value_data_2 = pd.DataFrame({'column_name': data.columns,
                                 'percent_missing': percent_missing})

#Algoritmo cria um vetor com a porcentagem de nulos para cada coluna e cria um
#novo dataframe com esses valores

missing_value_data_2 = missing_value_data_2.sort_values(by='percent_missing')

#Algoritmo ordena o dataframe pelo valor da porcentagem de nulos

In [133]:
#@title Algoritmo de criação de dicionário

#@markdown Execute o algoritmo para criar o dicionário de  valores a serem executados no modelo preditivo de regressão

columns_dict_1 = []
columns_dict_2 = []
default_data_2 = []
values = []
dropna_columns = []
default_columns = {}

#Criação de vetores e dicionário

for x in range(len(missing_value_data_2)):
  if missing_value_data_2["percent_missing"][x] == 0:
    columns_dict_1.append(missing_value_data_2["column_name"][x])
  elif missing_value_data_2["percent_missing"][x] <= 3:
    dropna_columns.append(missing_value_data_2["column_name"][x])
  elif missing_value_data_2["percent_missing"][x] <= 20:
    columns_dict_2.append(missing_value_data_2["column_name"][x])
  else:
    default_data_2.append(missing_value_data_2["column_name"][x])

print(dropna_columns)

data_dropna = data[dropna_columns].copy()

default_data_2.extend(dropna_columns)

data = data.drop(default_data_2, axis=1).copy()

#Este laço verifica os valores percentuais igual a 0 e armazena em um vetor e 
#os outros armazena em ordem em outro vetor

count = 0

values = columns_dict_1.copy()

for x in columns_dict_2:
  default_columns[x] = list(values)
  values.append(x)

#Este laço adicona no dicionário o vetor de colunas e a chave respectiva a este vetor
#para que seja possível percorrer cada predição com o conjunto de colunas para aquele target

['her2_ihc', 'recidive', 'age', 'escolari']


In [134]:
#@title Algoritmo de predição de dados para todas as colunas regresivas

#@markdown Execute o algoritmo para predizer e repopular a base restante com nulos por resultados da predição regressiva

for x in default_columns.keys():

  data_current = data.copy()
  data_current = data_current[default_columns.get(x)+[x]]

  data_nan_2 = data_current[data_current[x].isnull()]
  data_not_nan_2 = data_current[data_current[x].notnull()]

  regr = RandomForestRegressor(max_depth=2, random_state=0)

  regr.fit(data_not_nan_2[default_columns.get(x)], data_not_nan_2[x])

  data_nan_2[x] = regr.predict(data_nan_2[default_columns.get(x)])

  data[x] = pd.concat([data_nan_2[x], data_not_nan_2[x]])

#Algoritmo de predição por classificação para a chave x do dicionário com a 
#respectiva chave como base da entrada do modelo



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

In [135]:
#@title Merge dos dataframes e exclusão de linhas com nulos

data = data.merge(data_dropna, left_index=True, right_index=True)

data = data.dropna()


In [136]:
#@title Exclusão de colunas com apenas um dado único

drop_columns = []

for x in data.columns:
  if len(data[x].unique()) < 2:
    drop_columns.append(x)
data = data.drop(drop_columns, axis = 1).copy()

In [137]:
#@title Método Head com 100 linhas

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

data.head(100)

#Definição de linhas e colunas ilimitadas com a exibição das primeiras 100 linhas

,bmi,weight,height,contraceptive_method___1,antec_fam_cancer_mama,alcohol_type___1,alcohol_type___2,familial_degree___1,familial_degree___2,familial_degree___3,family_cancer_id___1,family_cancer_id___2,family_cancer_id___3,family_cancer_id___4,family_cancer_id___6,er_ihc,pr_ihc,tumor_subtype,follow_up_days,ultinfo,reclocal,recregio,recdist,trathosp,topo,basediag,morfo,tumor_stage,t,n,m,treatment_time,exam_time,her2_ihc,recidive,age,escolari
0,0.489334,-0.016035,152.100000,0.0,2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,3585.0,2.0,1.0,1.0,0.0,9.0,509.0,3.0,85003.0,10.0,11.0,0.0,0.0,4382.910000,1826.212500,1.0,1.0,27027.9450,2.0
1,1.302154,0.122837,157.909091,0.0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,3.0,2225.0,4.0,0.0,0.0,0.0,8.0,504.0,3.0,85003.0,21.0,2.0,0.0,0.0,1826.212500,1460.970000,4.0,0.0,20818.8225,2.0
2,4.553521,0.045902,155.117647,0.0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,3294.0,2.0,1.0,1.0,0.0,5.0,508.0,3.0,85003.0,31.0,3.0,1.0,0.0,2921.940000,4382.910000,1.0,1.0,26662.7025,4.0
3,0.605163,0.022619,152.000000,0.0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,3.0,4153.0,2.0,0.0,0.0,0.0,9.0,509.0,3.0,84803.0,21.0,2.0,0.0,0.0,3287.182500,1460.970000,4.0,0.0,25932.2175,2.0
4,0.523438,0.007120,167.000000,0.0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,3.0,3290.0,2.0,0.0,0.0,0.0,7.0,508.0,3.0,85003.0,22.0,2.0,1.0,0.0,2921.940000,1460.970000,4.0,0.0,20453.5800,3.0
5,0.953785,-0.138095,161.000000,0.0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,2224.0,3.0,0.0,1.0,0.0,4.0,504.0,3.0,85003.0,21.0,1.0,1.0,0.0,730.485000,730.485000,1.0,1.0,17896.8825,2.0
6,1.576012,0.114815,146.769231,0.0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,3.0,182.0,1.0,0.0,0.0,0.0,3.0,509.0,3.0,85003.0,21.0,13.0,1.0,0.0,589.794330,4382.910000,4.0,0.0,24106.0050,9.0
7,1.643721,-0.051724,149.333333,0.0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,3.0,3234.0,2.0,0.0,0.0,1.0,9.0,509.0,3.0,85003.0,22.0,2.0,1.0,0.0,2921.940000,1460.970000,4.0,1.0,19723.0950,9.0
8,1.196244,0.094505,154.000000,0.0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,428.0,3.0,0.0,1.0,0.0,3.0,509.0,3.0,85003.0,40.0,4.0,2.0,1.0,0.000000,365.242500,4.0,1.0,22279.7925,3.0
9,1.826017,-0.052392,158.764706,0.0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,3.0,700.0,2.0,0.0,0.0,0.0,9.0,509.0,3.0,85003.0,31.0,3.0,2.0,0.0,365.242500,1826.212500,4.0,0.0,15705.4275,2.0


#**Algoritmo de agrupamento**

In [138]:
#@title Configuração inicial

#Criando uma cópia do dataframe

data_classified = data.copy()

#Tirando a colunas que irão atrapalhar o algoritmo

data_classified = data_classified.drop(["treatment_time","ultinfo","alcohol_type___2",
       "familial_degree___1", "familial_degree___2", "familial_degree___3",
       "family_cancer_id___1", "family_cancer_id___2", "family_cancer_id___3",
       "family_cancer_id___4", "family_cancer_id___6"], axis=1)

data_classified.columns

Index(['bmi', 'weight', 'height', 'contraceptive_method___1',
       'antec_fam_cancer_mama', 'alcohol_type___1', 'er_ihc', 'pr_ihc',
       'tumor_subtype', 'follow_up_days', 'reclocal', 'recregio', 'recdist',
       'trathosp', 'topo', 'basediag', 'morfo', 'tumor_stage', 't', 'n', 'm',
       'exam_time', 'her2_ihc', 'recidive', 'age', 'escolari'],
      dtype='object')

In [139]:
#@title Execute o algoritmo para agrupar conjunto de valores numéricos

z = "follow_up_days"
if z in data.columns:
  for i in range(data[z].size):
    if data.iloc[i][z] <= data[z].quantile(1/4):
      value = 0
      data_classified[z].replace(data.iloc[i][z], str(value), inplace=True)
    elif data.iloc[i][z] <= data[z].quantile(7/10):
      value = 1
      data_classified[z].replace(data.iloc[i][z], str(value), inplace=True)
    else:
      value = 2
      data_classified[z].replace(data.iloc[i][z], str(value), inplace=True)
  data_classified[z] = data_classified[z].astype('int')


#**Seleção de features**

In [140]:
#@title Configuração inicial

#Criando uma cópia do dataframe

df = data_classified.copy()

In [141]:
#@title Selecionar melhor K-Best

#Definir a váriavel alvo, ou seja, o target
X = df.drop('follow_up_days', axis = 1).copy()
y = df['follow_up_days'] 

#Utilizar o método K-Best que é importada da biblioteca Sklearn
f_classif = SelectKBest(score_func = f_classif, k=22)
fit = f_classif.fit(X,y)

#Transformar em dataset
features = fit.transform(X)

#Transformar em dataset
cols = fit.get_support(indices=True)
df = df.iloc[:,cols ]

#Juntar o follow_up_days com o dataset
if "follow_up_days" not in df.columns:
  df = df.merge(df, y, left_index=True, right_index=True)

#Mostrar quais colunas estão no Dataset
df.columns

Index(['bmi', 'weight', 'height', 'antec_fam_cancer_mama', 'alcohol_type___1',
       'er_ihc', 'pr_ihc', 'tumor_subtype', 'follow_up_days', 'reclocal',
       'recregio', 'recdist', 'trathosp', 'morfo', 'tumor_stage', 't', 'n',
       'm', 'exam_time', 'her2_ihc', 'recidive', 'age'],
      dtype='object')

In [142]:
#@title Baixar base de dados

df.to_csv("/content/drive/Shareddrives/Colab_Grupo2/Colab/Data/dataFrameSelecionadoCopy.csv", index = False)